In [ ]:
from google.cloud import bigquery
from google.colab import drive
from google.colab import auth
import re
import os
import pandas as pd
drive.mount('/content/drive')
auth.authenticate_user()

project_id = 'iter-data-storage-pv-uat'
client = bigquery.Client(project_id)

Mounted at /content/drive


In [ ]:
path='/content/drive/MyDrive/Proyectos - IS/PROY - MAE/Validaciones/Validacion Datalake/validacion de plataforma raw/query_igualdad/'

In [ ]:
path='/content/drive/MyDrive/Proyectos - IS/PROY - MAE/Validaciones/Validacion Datalake/validacion de plataforma raw/query_null/'


In [ ]:
for archivo_sql in os.listdir(path):
  print(archivo_sql)

In [ ]:
def read_table_bq(path,archivo_sql):
  ruta_sql = os.path.join(path, archivo_sql)
  with open(ruta_sql, 'r') as file:
      sql = file.read()
  try:
    df_temp = pd.read_gbq(sql, project_id=project_id, dialect='standard')
    df = df_temp.melt(value_vars=[col for col in df_temp.columns if 'DIFF' in col],var_name='CAMPO_VALIDACION', value_name='RESULTADO')
    df['NOMBRE_TABLA']=archivo_sql
  except Exception as e:
    print(f'Error al ejecutar {archivo_sql} en BigQuery: {e}')
    with open('errores.log', 'a') as log_file:
      log_file.write(f'Error en {archivo_sql}: {e}\n')
      return None  # Devolver None para indicar que ocurrió un error
  return df

In [ ]:
df_concatenado = pd.DataFrame()
for archivo_sql in os.listdir(path):
  dk = read_table_bq(path,archivo_sql)
  df_concatenado = pd.concat([df_concatenado, dk], ignore_index=True)

In [ ]:
df_concatenado.to_excel('VALIDACION_TABLAS_BQ_NULL.xlsx')